In [1]:
# from syft_rds import *

In [2]:
from syft_rds.db.db import StorableItemRegistry
StorableItemRegistry.items


[syft_rds.models.base_model.Item,
 syft_rds.models.action_object.ActionObject,
 syft_rds.models.code.Code,
 syft_rds.models.dataset.Dataset,
 syft_rds.models.job.Job,
 syft_rds.models.request.Request]

In [3]:
from syft_rds.models.dataset import Dataset, DatasetCreate
from typing import Any

In [4]:


from __future__ import annotations
from syft_event import SyftEvents
from syft_rds.api.api import API
from syft_core import Client
from syft_rds.models.dataset import Dataset, DatasetCreate
from syft_rds.service.dataset_service import DatasetService
from syft_rds.connection.connection import get_connection
from syft_rds.service.context import BaseRPCContext
from syft_rds.models.job import JobCreate
from syft_rds.models.job import Job
from syft_rds.models.action_object import ActionObject
import shutil


In [5]:
from syft_rds.db.db import StorableItemRegistry
StorableItemRegistry.items


[syft_rds.models.base_model.Item,
 syft_rds.models.action_object.ActionObject,
 syft_rds.models.code.Code,
 syft_rds.models.dataset.Dataset,
 syft_rds.models.job.Job,
 syft_rds.models.request.Request]

In [6]:
client = Client.load()

In [7]:
# input policy: constrain the inputs of a syftfunction
# output policy: How do we modify the outputs of a syftfunction, how many outputs can you create

In [8]:
from uuid import UUID
from syft_core import SyftBoxURL
from syft_rds.models.code import CodeCreate, Code
from syft_rds.models.job import Job
from syft_rds.models.request import Request, RequestCreate
from syft_rds.service.code_service import CodeService
from syft_rds.service.job_service import JobService
from syft_rds.service.request_service import RequestService


box = SyftEvents("my-rds-app")

# TODO: pass this in the handler
context = BaseRPCContext(client=client, box=box)  


@box.on_request("/apis/list")
def get_apis() -> list[str]:
    """Respond to a ping request."""
    return {SyftBoxURL.from_path(k, client.workspace): v for k, v in box._SyftEvents__rpc.items()}

@box.on_request("/datasets/create")
def create_dataset(dataset: DatasetCreate) -> Dataset:
    """Respond to a ping request."""
    dataset_service = DatasetService.from_context(context)
    res = dataset_service.create_item(dataset)
    return res
    
@box.on_request("/datasets/list")
def list_datasets() -> list[Dataset]:
    """Respond to a ping request."""
    dataset_service = DatasetService.from_context(context)
    res = dataset_service.list_items()
    return res


@box.on_request("/code/create")
def create_code(code: CodeCreate) -> Code:
    """Respond to a ping request."""
    code_service = CodeService.from_context(context)
    res = code_service.create_item(code)
    return res


@box.on_request("/requests/create")
def create_request(request: RequestCreate) -> Request:
    """Respond to a ping request."""
    request_service = RequestService.from_context(context)
    res = request_service.create_item(request)
    return res

@box.on_request("/requests/approve")
def approve_request(request_id: str) -> Request:
    # TODO, fix serialization for UUIDs
    request_id = UUID(request_id)
    """Respond to a ping request."""
    request_service = RequestService.from_context(context)
    res = request_service.approve_request(request_id)
    return res

@box.on_request("/jobs/spawn")
def spawn_job(job: JobCreate) -> Job:
    # todo: maybe we dont need jobcreate and we can just pass in the args (need to fix serialization)
    """Respond to a ping request."""
    job_service = JobService.from_context(context)
    res = job_service.spawn_job(job)
    return res

@box.on_request("/jobs/get_result")
def get_job_result(job_id: str) -> ActionObject:
    job_id = UUID(job_id)
    job_service = JobService.from_context(context)
    res = job_service.get_job_result(context=context, job_id=job_id)
    return res

2025-02-14 10:41:35.410 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /apis/list
2025-02-14 10:41:35.411 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /datasets/create
2025-02-14 10:41:35.412 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /datasets/list
2025-02-14 10:41:35.413 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /code/create
2025-02-14 10:41:35.415 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /requests/create
2025-02-14 10:41:35.416 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /requests/approve
2025-02-14 10:41:35.417 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /jobs/spawn
2025-02-14 10:41:35.429 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /jobs/get_result


In [9]:
app_data_dir = client.my_datasite / "apps" / context.box.app_name
shutil.rmtree(app_data_dir, ignore_errors=True)

In [10]:
conn = get_connection(box, mock=True)   
api = API.from_email(client.email, conn)

In [11]:
api.datasets.create(body=DatasetCreate(name="my-dataset", description="my dataset"))

Dataset(uid=UUID('b48624f7-00b9-4ff1-86c4-1576bb44ca5c'), name='my-dataset', description='my dataset')

In [12]:
datasets = api.datasets.list()

In [13]:
dataset = datasets[0]

In [14]:
code = api.code.create(body=CodeCreate(code_str="my-code"))

writing to /Users/koen/Desktop/SyftBox/datasites/koen@openmined.org/apps/my-rds-app/items/SyftCode/d67426fb-d7cb-4dae-bd94-c5eff12cad05.json


In [15]:
request = api.requests.create(
    body=RequestCreate(code_id=code.uid,kwargs={"y": dataset.uid})
)


writing to /Users/koen/Desktop/SyftBox/datasites/koen@openmined.org/apps/my-rds-app/items/SyftRequest/d296d8f4-0fdc-4206-aac6-dc9473e0e450.json


In [16]:
request

Request(uid=UUID('d296d8f4-0fdc-4206-aac6-dc9473e0e450'), created_at=datetime.datetime(2025, 2, 14, 10, 41, 35, 497582), updated_at=datetime.datetime(2025, 2, 14, 10, 41, 35, 497587), code_id=UUID('d67426fb-d7cb-4dae-bd94-c5eff12cad05'), kwargs={'y': UUID('b48624f7-00b9-4ff1-86c4-1576bb44ca5c')}, status='pending')

In [17]:
api.requests.approve(body=str(request.uid)) #TODO: fix serialization for UUIDs

In [18]:
job = api.jobs.spawn(body=JobCreate(code_id=code.uid, kwargs={"y": dataset.uid}))


writing to /Users/koen/Desktop/SyftBox/datasites/koen@openmined.org/apps/my-rds-app/items/SyftJob/4058a9b7-fc5a-4172-8567-7ef1ed92cbb6.json


In [19]:
job

Job(uid=UUID('4058a9b7-fc5a-4172-8567-7ef1ed92cbb6'), created_at=datetime.datetime(2025, 2, 14, 10, 41, 35, 638848), updated_at=datetime.datetime(2025, 2, 14, 10, 41, 35, 638852), code_id=UUID('d67426fb-d7cb-4dae-bd94-c5eff12cad05'), kwargs={'y': UUID('b48624f7-00b9-4ff1-86c4-1576bb44ca5c')}, status='pending', result_id=UUID('a80d8632-7643-4d9c-8649-04801d311791'))

## Run the job (mock executor)

In [20]:

class Executor:
    def __init__(self, client: Client, context: BaseRPCContext):
        self.client = client
        self.context = context
        
    def write_result(self, res_obj: ActionObject):
        res_path = self.context.item_dir(ActionObject) / f"{res_obj.uid}.json"
        res_path.parent.mkdir(parents=True, exist_ok=True)
        res_path.write_text(res_obj.model_dump_json())

    def execute_code(self):
        job_files = list((self.context.item_dir(Job)).glob("*.json"))
        job = Job.model_validate_json(job_files[0].read_text())
        code_id = job.code_id
        code_files = self.context.item_dir(Code).glob("*.json")
        code_file = [c for c in code_files if c.name == f"{code_id}.json"][0]
        code = Code.model_validate_json(code_file.read_text())
        # execute code here
        res = 1
        res_obj = ActionObject(syft_action_data=res, uid=job.result_id)
        self.write_result(res_obj)

In [21]:
executor = Executor(client, context)
executor.execute_code()


# Back to client side

In [28]:
api.jobs.get_result(body=str(job.uid))


TypeError: JobService.get_job_result() missing 1 required positional argument: 'job_id'

In [22]:
job.result_id

UUID('a80d8632-7643-4d9c-8649-04801d311791')

# Appendix

In [23]:
res = conn.send(
    url=f"syft://{client.email}/api_data/my-rds-app/rpc/apis/list",
    body={},
    expiry="5m",
    cache=True,
)

In [24]:
res

{'syft://koen@openmined.org/api_data/my-rds-app/rpc/apis/list': <function __main__.get_apis() -> 'list[str]'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/datasets/create': <function __main__.create_dataset(dataset: 'DatasetCreate') -> 'Dataset'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/datasets/list': <function __main__.list_datasets() -> 'list[Dataset]'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/code/create': <function __main__.create_code(code: 'CodeCreate') -> 'Code'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/requests/create': <function __main__.create_request(request: 'RequestCreate') -> 'Request'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/requests/approve': <function __main__.approve_request(request_id: 'str') -> 'Request'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/jobs/spawn': <function __main__.spawn_job(job: 'JobCreate') -> 'Job'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/jobs/get_result': <function __m

In [25]:
res = conn.send(
    url=f"syft://{client.email}/api_data/my-rds-app/rpc/datasets/create",
    body=DatasetCreate(name="my-dataset", description="my dataset"),
    expiry="5m",
    cache=True,
)

In [26]:
res

Dataset(uid=UUID('7cfdcd7a-b59e-413f-b74b-a9d287a98533'), name='my-dataset', description='my dataset')